In [ ]:

def stock_list_annotation(tickers, client,nation):
    """Get top N intraday losers from a predefined S&P 500 ticker list with company details
    
    Returns DataFrame with company name, sector, market cap, current price, and percentage change.
    Includes random sleep intervals between API calls to prevent rate limiting.
    """
    losers = []
    ending=nation_ending[nation]
    def get_market_cap_formatted(market_cap):
        """Convert market cap to readable format"""
        if not market_cap:
            return 'N/A'
        billion = 1_000_000_000
        if market_cap >= billion:
            return f"${round(market_cap/billion, 1)}B"
        else:
            return f"${round(market_cap/1_000_000, 1)}M"
    
    for ticker in tickers:
        print(ticker)
        try:
            #Random sleep between 0 and 3 seconds
            


            stock = yf.Ticker(ticker+ending)
            data = stock.history(period='1d', interval='1m')
            dic={
                'Ticker': ticker,
                'Company Name': stock.info.get('longName', 'N/A'),
                'Sector': stock.info.get('sector', 'N/A'),
                'Market Cap': get_market_cap_formatted(stock.info.get('marketCap', None)),
            }

            dic['Summary']=summary(dic['Company Name'],nation,client)
            print(dic['Summary'])
            
        except Exception as e:
            dic=dict()
        losers.append(dic)
        sleep_time=np.random.gamma(4,5)
        print(sleep_time)
        time.sleep(sleep_time)
    losers_df = pd.DataFrame(losers)
    if len(losers_df) == 0:
        return pd.DataFrame(columns=['Ticker', 'Company Name', 'Sector', 'Market Cap', 'Summary'])
    
    #losers_df = losers_df.sort_values(by='% Change')
    return losers_df[['Ticker', 'Company Name', 'Sector', 'Market Cap', 'Summary']].reset_index(drop=True)
